In [11]:
%pip install torch zkstats==0.1.6


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import json
import os
from zkstats.core import (
    prover_gen_settings,
    prover_gen_proof,
    setup,
)
from zkstats.computation import computation_to_model

cwd = os.getcwd()

# FIXME: fill this in with the path to your data
data_path = f"{cwd}/data.json"

# Paths to the output files
output_dir = f"{cwd}/out"
os.makedirs(output_dir, exist_ok=True)
model_onnx_path = f"{output_dir}/model.onnx"
compiled_model_path = f"{output_dir}/model.compiled"

pk_path = f"{output_dir}/model.pk"
vk_path = f"{output_dir}/model.vk"
proof_path = f"{output_dir}/model.pf"
settings_path = f"{output_dir}/settings.json"
witness_path = f"{output_dir}/witness.json"
comb_data_path = f"{output_dir}/comb_data.json"
precal_witness_path = f"{output_dir}/precal_witness.json"

In [13]:
# Data provided shared by the data provider beforehand

def get_data_shape(data_path: str) -> dict[str, int]:
    """
    Get the shape of the data from the data file.

    Parameters:
    - data_path (str): The path to the data file.

    Returns:
    - shape_info (dict): A dictionary where keys are column names and values are the number of elements (shape).
    """
    with open(data_path, 'r') as f:
        data = json.load(f)
    shape_info = {col: len(data[col]) for col in data.keys()}
    return shape_info

data_shape = get_data_shape(data_path)
data_commitment_path = f"{output_dir}/data_commitment.json"

## Columns selected by the user

In [14]:
# FIXME: this should be provided by users
selected_columns = ["x", "y"]

## User-defined Computation

A computation should be of type `TComputation`. For example, the following code snippet defines a computation that computes the sum of the private data.

```python
def computation(state: State, x: list[torch.Tensor]):
    out_0 = state.median(x[0])
    out_1 = state.median(x[1])
    return state.mean(torch.cat([out_0.unsqueeze(0), out_1.unsqueeze(0)]).reshape(-1,1))
```

FIXME: The following code snippet is entirely from the user. You MUST check
1. the code only performs zkstats-related operations.
2. the computation must not leak any information about the private data.

In [15]:
# This is just a dummy computation. Replace it with user's computation
import torch
from zkstats.computation import State

def computation(state: State, x: list[torch.Tensor]):
    out_0 = state.median(x[0])
    out_1 = state.median(x[1])
    return state.mean(torch.cat([out_0.unsqueeze(0), out_1.unsqueeze(0)]).reshape(-1,1))

Generate settings and setup with user's computation.

In [16]:
_, model = computation_to_model(computation, precal_witness_path, isProver=True)
prover_gen_settings(
    data_path,
    selected_columns,
    comb_data_path,
    model,
    model_onnx_path,
    [3],
    "resources",
    settings_path,
)

# Determine which srs to use with the logrows in the settings.json
with open(settings_path, "r") as f:
    settings = json.load(f)
logrows = int(settings["run_args"]["logrows"])
srs_path = f'~/.ezkl/srs/kzg{logrows}.srs'

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 3, param_scale: 3, scale_input_multiplier: 10) ------------->

+--------------+--------------+-----------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error   | median_error | max_error | min_error    | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+--------------+--------------+-----------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.012499809 | -0.024999619 | 0         | -0.024999619 | 0.012499809    | 0.024999619      | 0.024999619   | 0             | 0.00031249048      | -0.0005030104      | 0.0005030104           |
+--------------+--------------+-----------+--------------+----------------+------------------+---------

scale:  [3]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":3,"param_scale":3,"scale_rebase_multiplier":10,"lookup_range":[-856,754],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":2624,"total_assignments":950,"total_const_size":421,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,3],"model_input_scales":[3,3],"module_sizes":{"kzg":[],"poseidon":[2624,[2]]},"required_lookups":["ReLU","Abs",{"GreaterThan":{"a":0.0}},{"Floor":{"scale":16.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1718381959531}


In [17]:
setup(model_onnx_path, compiled_model_path, settings_path, vk_path, pk_path)

==== setting up ezkl ====
Time setup: 0.8122658729553223 seconds


Generate proof with your data and user's computation.

In [18]:
prover_gen_proof(
    model_onnx_path,
    comb_data_path,
    witness_path,
    compiled_model_path,
    settings_path,
    proof_path,
    pk_path,
)

==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 24.875
==== Generating Proof ====
proof:  {'instances': [['3bea079e2a500a3285d05fd174263a5ffd78c2dd63131c52abd2a199f786f601', '8ef07c3e2daefe521c14b14bc10f91b50ea0d110db70b3192443c6f25d1cd124', '0100000000000000000000000000000000000000000000000000000000000000', 'c700000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x29c7f676f3fa24cc43a4c912c803cdbc2415937a79646a88743ee9de551a445e2733c2f93b0afd938aa2d8309af9610cd94ad74f2c65cfc77e0a7ff28ef1be85063645219291ed6931e3dbdc915684bb1fd9402bc3f6abac6e2541777a7181e7145f4f902b671464277ecfc4cbe1235c5e4fb302e2ea7fb4881d585adfa162ff0c2f82d2bde6cc427da130b8fc413467d1d6fbeb5c848dae899367d243973b4d26126ea881a75bc88275abf808dca5a42a2f9e89888ca8f6d54d8081d6a21e2a2aeaf6b92121f2b14e7527b6737cf6d27c4728123a42edb69546befa21cc547500d4b86b850e29dffe5e874eddabf664f084b0c9a6b9a7b6ccd78d222d3509ff2b993a9349f846b431e912bbbe7e9d5d57a3ba35552096e93ddb7cf44342ff74069534f809

## Verify the proof to ensure it is correct
NOTE: The following section is to illustrate what should be done on the user (data consumer) side. This step is not required by the data provider.

In [19]:
import json
import numpy as np
from typing import Dict


def create_dummy(shape_info: Dict[str, int], dummy_data_path: str) -> None:
    """
    Create a dummy data file with randomized data based on the provided shape information.

    Parameters:
    - shape_info (dict): A dictionary where keys are column names and values are the number of elements (shape).
    - dummy_data_path (str): The path to save the dummy data file.
    """
    dummy_data = {}
    for col, length in shape_info.items():
        # Generate random data for each column
        dummy_data[col] = np.round(np.random.uniform(0, 100, length), 1).tolist()

    with open(dummy_data_path, 'w') as f:
        json.dump(dummy_data, f)


from zkstats.core import verifier_define_calculation, verifier_verify

verifier_model_path = f"{output_dir}/verifier_model.onnx"
verifier_compiled_model_path = f"{output_dir}/verifier_model.compiled"
verifier_vk_path = f"{output_dir}/verifier_model.vk"
verifier_pk_path = f"{output_dir}/verifier_model.pk"
dummy_data_path = f"{output_dir}/dummy_data.json"
sel_dummy_data_path = f"{output_dir}/sel_dummy_data.json"

# NOTE: generate the verifier model with the `precal_witness_path` provided by the prover
_, verifier_model = computation_to_model(computation, precal_witness_path, isProver=False)
# create dummy data with the same shape as the original data
create_dummy(data_shape, dummy_data_path)
# generate the verifier model given the dummy data and the selected columns
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)
# generate the verification key
setup(verifier_model_path, verifier_compiled_model_path, settings_path, verifier_vk_path, verifier_pk_path)
# verify the proof
verifier_verify(proof_path, settings_path, verifier_vk_path, selected_columns, data_commitment_path)

==== setting up ezkl ====
Time setup: 0.8539810180664062 seconds


[24.875]

Print the file paths. You should share the following files back to the user for them to verify the proof. You **SHOULD NOT** share more files otherwise data might be leaked.

In [20]:
print("Model onnx:\t\t", model_onnx_path)
print("Settings:\t\t", settings_path)
print("Proof:\t\t\t", proof_path)
print("Verification key:\t", vk_path)
print("Srs path:\t\t", srs_path)

Model onnx:		 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.onnx
Settings:		 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/settings.json
Proof:			 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.pf
Verification key:	 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.vk
Srs path:		 ~/.ezkl/srs/kzg12.srs
